# Technical Evaluation Master Notebook (Ilyas ADEN)

## 1. Introduction

In this notebook, we will describe necessary steps for completing given task. in order to address current use case of comparing **two legal documents** (Apple Music's Terms and Conditions for year 2015 and 2023), we have decided to implement a **retrieval-augmented generation (RAG)** solution. We walk through RAG using the new gpt-index library called now LlamaIndex. This new library will help to mitigate response failure, and provide a comprehensive pipelines for RAG implementation.
In this document, we will outline the 5 stages for implementing this RAG pipeline described as below:

             Loading--> Indexing --> Storing --> Querying --> Evaluation 

1. Loading involves fetching data from different sources and integrating it into your pipeline using LlamaHub connectors (ideal for csv, pdf, api,google docs files), however as current version of LLamainex 0.9.4 did not allow us to load .docx, we have transformed current given word documents to pdf extension.
2. Indexing creates structured representations, including vector embeddings and metadata, for efficient data retrieval in LLMs.
3. Storing the indexed data and metadata prevents the need for re-indexing, Llamaindex have its own vector database but in production we can use Pinecone.
4. Querying involves using LLMs and LlamaIndex data structures for various query strategies. we will test a **basic** query engine and a **sub-questions-query** engine.
5. Evaluation measures the pipeline's effectiveness, offering objective metrics for response accuracy, faithfulness, and speed. we will compare the two strategies.

Sources:
- https://docs.llamaindex.ai/en/stable/getting_started/concepts.html
- https://docs.llamaindex.ai/en/stable/examples/retrievers/recursive_retriever_nodes.html


# 2. Importing libraries

In [1]:
# importing necessary libraries
from llama_index import SimpleDirectoryReader, ServiceContext, VectorStoreIndex
from llama_index.llms import OpenAI

from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

In [2]:
# Checking the python environment

import os
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
# Setting the OpenAI api key for once.
import openai
import getpass  
# we get API key on OpenAI website for our current test, please use "sk----"
openai.api_key = os.getenv("OPENAI_API_KEY") or getpass.getpass("Enter your OpenAI API key: ")


# 3. RAG Implementation 
## 3.1 Loading stage

**Parsing the documents into text Chunks (Nodes) in Llamaindex, we use a single line of code**

In [4]:

documents=SimpleDirectoryReader("/Users/ilyasaden/myrepo/Simmons_Tech_Challenge/data/").load_data()

In [5]:
documents

[Document(id_='53690a00-d68f-4b5a-b51f-7c1931fe0575', embedding=None, metadata={'page_label': '1', 'file_name': 'JAN2015.pdf', 'file_path': '/Users/ilyasaden/myrepo/Simmons_Tech_Challenge/data/JAN2015.pdf', 'file_type': 'application/pdf', 'file_size': 229026, 'creation_date': '2024-02-04', 'last_modified_date': '2024-02-04', 'last_accessed_date': '2024-02-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='TERMS AND CONDITIONS A. ITUNES STORE, MAC APP STORE, APP STORE, AND IBOOKS STORE TERMS OF SALE B. ITUNES STORE TERMS AND CONDITIONS C. MAC APP STORE, APP STORE AND IBOOKS STORE TERMS AND CONDITIONS  THE LEGAL AGREEMENTS SET OUT BELOW ARE BETWEEN YOU AND ITUNES SARL (“ITUNES") AND GOVERN YOUR USE OF THE ITUNES STORE, MAC APP STORE, APP STORE AND IBOOK

## 3.2 Selecting LLM model

In [6]:
llm = OpenAI(temperature=0, model="gpt-4")
service_context = ServiceContext.from_defaults(llm=llm)

## 3.3 Indexing, storage Stages (in LlamaIndex is a single line)

In [7]:
index=VectorStoreIndex.from_documents(documents,show_progress=True)

/Users/ilyasaden/myrepo/Simmons_Tech_Challenge/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 46/46 [00:01<00:00, 26.59it/s]


In [8]:
# Checking index
index

## 3.4 Querying Stage

We sets up a a retriever query engine, a post-processor for similarity-based filtering for top k=4 and only similarity of 80% as criteria. 

In [9]:
query_engine=index.as_query_engine()

In [10]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.80)

query_engine=RetrieverQueryEngine(retriever=retriever,
                                  node_postprocessors=[postprocessor])


In [11]:
response=query_engine.query("What is apple music's terms and conditions in 2023 ?")

In [12]:
from llama_index.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
print(response)

Final Response: Apple Music's terms and conditions in 2023 include the
following: - iCloud Music Library is an Apple Music feature that
allows you to access your matched or uploaded songs, playlists, and
music videos. - iCloud Music Library collects information about your
iCloud Music Library Content, which is associated with your Apple ID.
- You can upload up to 100,000 songs to iCloud Music Library, and
songs acquired from the iTunes Store do not count against this limit.
- When you use iCloud Music Library, Apple logs information such as
the tracks you play, stop or skip, the devices you use, and the time
and duration of playback. - iCloud Music Library is provided on an "AS
IS" basis and could contain errors or inaccuracies. - If you are not
an Apple Music member, you may purchase an iTunes Match subscription,
which uses iCloud Music Library. - When your Apple Music membership
ends, you will lose access to your iCloud Music Library, including
iCloud Music Library Content that is up

**We can see that we have 4 sources nodes**

**Below examples of content of first two nodes**

In [13]:
from IPython.display import HTML, display

def set_css(ei):
    display(HTML('''
    <style>
        pre {
            white-space: pre-wrap;
        }
    </style>
    '''))

get_ipython().events.register('pre_run_cell', set_css)


In [14]:
#from IPython.display import HTML, display
from IPython.core.display import HTML

HTML("<pre>" + documents[0].get_content() + "</pre>")

In [15]:
#from IPython.display import HTML, display
from IPython.core.display import HTML

HTML("<pre>" + documents[1].get_content() + "</pre>")


## 4. RAG Assessments

### 4.1 Setting a baseline query engine

To confirm initial output, we decide to first setup a top-k query engine and check the results again using below code.

In [16]:
# setup baseline index
base_index = VectorStoreIndex.from_documents(documents, show_progress=True)
base_engine = base_index.as_query_engine(similarity_top_k=4)

Generating embeddings: 100%|██████████| 46/46 [00:01<00:00, 27.90it/s]


In [17]:
response = base_engine.query("What are some terms and conditions in 2015?")
print(str(response))

Some terms and conditions in 2015 include the requirement that only persons aged 13 years or older can create accounts, the need for compatible devices, internet access, and certain software to use the service, and the responsibility of the user to maintain the confidentiality and security of their account information. Additionally, the agreement states that iTunes reserves the right to modify the agreement at any time and to impose new or additional terms or conditions on the use of the service.


In [20]:
from IPython.core.display import HTML

HTML("<pre>" + str(response) + "</pre>")


In [21]:
print(len(response.source_nodes))

4


**Based on above query results, Let check every source node using below codes and see if we can find this particular output.**

In [22]:

from IPython.core.display import HTML

HTML("<pre>" + response.source_nodes[0].get_content() + "</pre>")

In [23]:
from IPython.core.display import HTML

HTML("<pre>" + response.source_nodes[1].get_content() + "</pre>")

**BINGO!! This second Node contains results retrieved, see line 5!!** 

**LET ASK A MORE COMPLICATE QUESTION WHICH IS THE MAIN OBJECTIVE OF THIS ASSESSMENTS**

In [24]:
response = base_engine.query("Compare and contrast the terms and conditions in 2015 and 2023?")
print(str(response))

I'm sorry, but I cannot compare and contrast the terms and conditions in 2015 and 2023 based on the given context information. The provided text only includes the terms and conditions for Apple's Services, but it does not provide any information about the terms and conditions in 2015 or any changes that may have occurred between 2015 and 2023.


In [25]:
from IPython.core.display import HTML

HTML("<pre>" + str(response) + "</pre>")

THE BASE ENGINE QUERY FAILED TO DELIVER LET SEE IF THE "SUBQUESTIONQUERY" ENGINE CAN HELP

### 4.2 Setting a "SubQuestionQueryEngine" to improve performance 
**This query engine perform following tasks:**

- Treat different documents as different "tools", we need to separate them during the loading and avoid storing as single collection of all documents.
- Breaks down any complex question into sub-questions over any subset of different documents
- Perform retrieval independently per document
- Aggregate results at the end.

In [26]:
jan2015_doc = SimpleDirectoryReader(input_files=["/Users/ilyasaden/myrepo/Simmons_Tech_Challenge/data/JAN2015.pdf"]).load_data()
mar2023_doc = SimpleDirectoryReader(input_files=["/Users/ilyasaden/myrepo/Simmons_Tech_Challenge/data/MAR2023.pdf"]).load_data()

In [27]:
jan2015_index = VectorStoreIndex.from_documents(jan2015_doc)

In [28]:
mar2023_index = VectorStoreIndex.from_documents(mar2023_doc)

In [29]:
jan2015_engine = jan2015_index.as_query_engine(similarity_top_k=2)

In [30]:
mar2023_engine = mar2023_index.as_query_engine(similarity_top_k=2)

In [31]:
# We create sub query as below:
query_engine_tools = [
    QueryEngineTool(
        query_engine=jan2015_engine,
        metadata=ToolMetadata(
            name="jan2015",
            description="Provides information about Apple Music's Terms and Conditions for year 2015",
        ),
    ),
    QueryEngineTool(
        query_engine=mar2023_engine,
        metadata=ToolMetadata(
            name="mar2023",
            description="Provides information about Apple Music's Terms and Conditions for year 2023",
        ),
    ),
]

s_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

### 5. Results analysis

We will compare our SubQuestionQueryEngine against the base query engine by asking the same question order.

In [33]:
import nest_asyncio

nest_asyncio.apply()

In [34]:
from IPython.core.display import HTML
response = s_engine.query("Compare and contrast Terms and Conditions in 2015 and 2023?")
#print(str(response))


HTML("<pre>" + (str(response)) + "</pre>")

Generated 2 sub questions.
[jan2015] Q: What are the Terms and Conditions for Apple Music in 2015?
[mar2023] Q: What are the Terms and Conditions for Apple Music in 2023?
[jan2015] A: The Terms and Conditions for Apple Music in 2015 are not provided in the given context information.
[mar2023] A: The Terms and Conditions for Apple Music in 2023 include information about iCloud Music Library, which is a feature that allows users to access their matched or uploaded songs, playlists, and music videos. It mentions that iCloud Music Library collects information about the user's content and is associated with their Apple ID. It also states that iCloud Music Library is provided on an "AS IS" basis and could contain errors or inaccuracies. Additionally, it mentions that when an Apple Music membership ends, the user will lose access to their iCloud Music Library.


** While the base query engine couldn't generate a detailed proper response, here we can see that the response is much more detailed Sub-questions query engine**

** And below one more comparison showing how base engine failed to return results**

In [36]:
query_str = "what are the main risks for customers between these two terms and conditions?"

base_response = base_engine.query(query_str)
response = s_engine.query(query_str)

Generated 2 sub questions.
[jan2015] Q: What are the main differences between the terms and conditions of jan2015 and mar2023?
[mar2023] Q: What are the main differences between the terms and conditions of jan2015 and mar2023?
[jan2015] A: The main differences between the terms and conditions of JAN2015 and MAR2023 cannot be determined based on the given context information.
[mar2023] A: There is no information provided in the context about the terms and conditions of Jan2015 or any specific differences between the terms and conditions of Jan2015 and Mar2023.


In [37]:
print(f"Base query engine: {str(response)}\n\n")
print(f"Sub-question query engine: {str(base_response)}\n\n")

Base query engine: The main risks for customers between the terms and conditions of Jan2015 and Mar2023 cannot be determined based on the given context information.


Sub-question query engine: The main risks for customers between these two terms and conditions are the potential loss of income, business, or profits, as well as the loss or corruption of data. Additionally, customers may be liable for any claims arising from a breach of the agreement. It is important for customers to be aware that iTunes does not provide any warranties or guarantees regarding the uninterrupted or error-free use of their services, and they may remove or suspend the service at any time for technical or operational reasons.




**After we reformulate the questions in a more suitable way, we see the base engine returns some results but SUb question query engine is more rich:**

In [38]:
query_str = "Compare and contrast Terms and Conditions for year 2015 and year 2023"

base_response = base_engine.query(query_str)
response = s_engine.query(query_str)

Generated 2 sub questions.
[jan2015] Q: What are the Terms and Conditions for Apple Music in 2015?
[mar2023] Q: What are the Terms and Conditions for Apple Music in 2023?
[jan2015] A: The Terms and Conditions for Apple Music in 2015 are not provided in the given context information.
[mar2023] A: The Terms and Conditions for Apple Music in 2023 include information about iCloud Music Library, which is a feature that allows users to access their matched or uploaded songs, playlists, and music videos. It collects information about the user's iCloud Music Library Content and is associated with their Apple ID. The Terms and Conditions also state that iCloud Music Library is provided on an "AS IS" basis and could contain errors or inaccuracies. Additionally, it is mentioned that users should back up their data and information before using iCloud Music Library.


In [39]:
print(f"Base query engine: {str(response)}\n\n")
print(f"Sub-question query engine: {str(base_response)}\n\n")

Base query engine: The Terms and Conditions for Apple Music in 2015 and 2023 differ in several aspects. In 2015, the specific Terms and Conditions for Apple Music are not provided in the given context information. However, in 2023, the Terms and Conditions mention the inclusion of iCloud Music Library as a feature. This feature allows users to access their matched or uploaded songs, playlists, and music videos. The Terms and Conditions also highlight that iCloud Music Library is provided on an "AS IS" basis and may contain errors or inaccuracies. Additionally, users are advised to back up their data and information before using iCloud Music Library. These differences indicate that there have been updates and changes to the Terms and Conditions of Apple Music between 2015 and 2023.


Sub-question query engine: The Terms and Conditions for the year 2015 and the year 2023 have some similarities and differences. In both years, the Terms and Conditions govern the use of Apple's Services and